In [1]:
import random

# Introduction

Bob, knows a given value `x`, and wants to prove to Alice that he knows `x`, without revealing to Alice what `x` is.


In [2]:
#Privately known variables (only known by Bob)
x = 6
x

6

# Part A - Bob is an Honest Actor

For Part A, assume Bob is an honest actor, and actually does know the value of `x`.

## 1. Public One-Way Function

Our example begins with a publicly known, one-way function.

A common example of a one-way function are hashes.

Suppose `f(x)` is any hash function, such as SHA-256, applied to the input of `x`. It is easy to take a given value of `x` and produce the hash: `f(x)`.

However, it is very difficult to take the hash, `f(x)`, and solve for the pre-hashed input `x`. In practice, the function can only be performed in one-way.

For more on hashes: https://en.wikipedia.org/wiki/Cryptographic_hash_function

---

In our example, we'll use `f(x) = (g ** x) % p` as another one-way function.

When given values for `x`, `g`, and `p`, it is simple to calculate `f(x)`.

But when given `f(x)`, `g`, and `p`, and asked to solve for `x`, there is no quick answer.

The one-way nature of this function is based on the discrete logarithm problem. There is no known efficient method to compute discrete logarithms in general (i.e. to solve for `x` in this equation). Solving for `x` requires resorting to a brute force method, guessing different values of `x` until one works.

For more on discrete logarithm problem: https://en.wikipedia.org/wiki/Discrete_logarithm

---

In Python we can express the syntax `(g ** x) % p` as simply `pow(g, x, p)`.

The function `pow()` is built in to Python, no imports required, and is used in practice because it is more efficient for large numbers than separately performing the power and modulus operations.

`pow(g, x, p) == (g ** x) % p`


## 2. Public Inputs Other Than `x`

The values `g` and `p` are also publicly known, since all that matters is that `x` is kept secret, so that Bob can prove he knows `x` without `x` itself being revealed.

The security in this case does not rely on keeping `g` or `p` secret, it relies on the computational difficulty of the discrete logarithm problem.

In other words, even when given `g`, `p`, and the ouput `pow(g, x, p)`, it's computationally difficult (virtually impossible) to solve for `x`.

---

In a real-world scenario, to ensure appropriate levels of security, the values for `g` and `p` would follow certain criteria:
- `p`:  a very large prime number. All else equal, the larger the prime number, the harder it is to brute force a solution
- `g`: a chosen primitive root modulo `p`. This property also ensures a large enough problem space, making it secure against brute-force attacks

For more on primitive root modulo n: https://en.wikipedia.org/wiki/Primitive_root_modulo_n

In this tutorial, to keep things simple, we'll abide by these same contraints but use a much, much smaller value for `p` than would be advisable for a real-world situation.

In [3]:
#Publicly known variables (known by Bob, Alice, and everyone else)
p = 23 # must be prime
g = 5 # must be primitive root modulo p


## 3. Bob Sends Result `fx` to Alice

To begin the process of proving that he knows the value of `x` to Alice, without revealing the value of `x` itself, Bob sends the output of the one-way function to Alice.

In other words, Alice now knows:
- `g`
- `p`
- `fx`

But still does not know `x`, because of the one-way nature of `fx = pow(g, x, p)`.

In [4]:
fx = pow(g, x, p)
fx

8

## 4. Bob Privately Picks a Random Integer `r`

Bob privately/securely generates a random integer bewteen 0 and `p-1`, called `r`. He does *not* share `r` with Alice, or anyone else.


In [5]:
def pick_private_random_int(p):
    r = random.randint(0, p - 1)
    return r

r = pick_private_random_int(p)
r

22

## 5. Bob Calculates `u` and Sends to Alice

Bob applies the public one-way function, replacing `x` with the random number `r`, and sends the result, `u`, to Alice.

It's important to note that Bob submits a given value of `u` to Alice *before* Alice generates a challenge in the next step.

In Part B, we'll see how this order of steps is critical to validating that Bob is being honest about knowing `x`.

In [6]:
def calculate_u(g, r, p):
    u = pow(g, r, p)
    return u

u = calculate_u(g, r, p)
u

1

## 6. Alice Generates a Challenge `e`

Alice randomly generates a challenge, `e`, as either 0 or 1 and sends it to Bob.

Again, as mentioned in Step 5 above, it's significant that this challenge occurs *after* Bob has already submitted a value of `u` to Alice.

In [7]:
def generate_challenge():
    e = random.randint(0, 1)
    return e

## 7. Bob Responds With `v` Depending on the Challenge

Bob responds with `v`, which depends on the challenge `e` from Alice. You may notice that `v` is only a function of `x` when `e` is 1.

In other words, whenever `e` is 0, Bob doesn't need to know `x` to return `v`. He could be lying about knowing `x`, so why bother ever using `e` of 0?

The reason to allow `e` to be 0 is that if both Bob knew ahead of time that `e` will always be 1, it's possible he could submit values of `u` and `v` that would look correct, just by knowing `f(x)`, even if he doesn't know `x` itself.

We'll see this play out in Part B in more detail. For now, keep in mind that the difficult to anticipate what `e` will be means Bob can not fake a valid response with a 100% chance of success.

In [8]:
def respond_to_challenge(e,r,x):
    if e == 1:
        v = r + x
    else:
        v = r
        
    return v

## 8. Alice Verifies the Proof

Alice can then verify the proof, based on the value of `e` she provided.

---

If `e` is 0, she asserts that the `u` value Bob provided is equal to `pow(g, v, p)`.

The reason for this is straightforward. Since when `e` is 0, if Bob is honest, he will have responded with `v = r`.

Thus `u = pow(g, v, p) = pow(g, r, p)`.

If Bob is honest, then he will have published a value of `u` calculated as `u = pow(g, r, p)`, so Alice can easily ensure that the values of `u` and `v` which Bob provided do indeed work out.

---

If `e` is 1, the math is a bit more complex.

Why should `(u * fx) % p = pow(g, v, p)`?

If Bob is honest, `u` is calculated as `pow(g, r, p)` and `fx` is calculated as `pow(g, x, p)`.
 
We can therefore rewrite `(u * fx) % p` as:

`(pow(g, r, p) * pow(g, x, p)) % p`

And without getting too deep into the mathematics itself, we can rewrite this simplified as:

`pow(g, r + x, p)`

Recall though, that in the case when `e` is 1, if Bob is honest, `v = r + x`, so we express this as:

`pow(g, v, p)`

Thus, when `e` is 1 and if Bob is honest, `(u * fx) % p` is just another way to write `pow(g, v, p)`, so if these two expressions are not equivalent, then Bob is not being honest.


In [9]:
def verify_proof(e,u,g,v,p,fx):
    if e == 0:
        assert u == pow(g, v, p), "Failed when e=0, we caught Bob lying about knowing x!"
    else:
        assert (u * fx) % p == pow(g, v, p), "Failed when e=1, we caught Bob lying about knowing x!"


## 9. Repeat Steps 4-8 Multiple Times

Steps 4-8 are repeated multiple times, because if only conducted once, it is possible that Bob is a bad actor and just happened to correctly guess whether `e` will be 0 or 1, and prepared a response that *always* works if `e` is 0 (or a response that *always* works if `e` is 1).

In other words, if only conducted once, there is a 50% chance that Bob is lying and he just happened to guess `e` correctly, so there is a 50% chance he can get away with appearing to know `x` even though he only knows `f(x)`. Conceptually, this would be like faking he knows a private key, while only actually knowing a public key.

If this process is repeated 10 times though, there is only a `0.5 ** 10` chance (less than a 0.1% chance) of him correctly guessing `e` and lying each time. The process can be repeated any number of times until Alice feels confident enough that Bob has proved to her that he is being honest about knowing `x`.


In [10]:
for i in range(0,10):
    r = pick_private_random_int(p)
    u = calculate_u(g, r, p)
    e = generate_challenge()
    v = respond_to_challenge(e,r,x)
    verify_proof(e,u,g,v,p,fx)
    
print('No assert failures! Bob is honest!')

No assert failures! Bob is honest!


What we've done so far is construct an "interactive proof", since it requires Bob and Alice to go back and forth several times for Alice to have confidence in Bob's proof that he knows `x`.

You may have heard of the term SNARK, which stands for "Succinct Non-interactive ARguments of Knowledge".

Notice how SNARKs are "non-interactive". The proof we have constructed so far is a zero knowledge proof, but it is not a SNARK in part because it is interactive. We'll look at making this non-interactive in Part C.


# Part B: Bob is a Bad Actor

Before making our proof non-interactive, we still have to answer the question: "what if Bob is lying?"

Perhaps Bob does not actually know `x`, the private secret, but knows `fx`, the public output of `pow(g, x, p)`.

In this case, it's possible Bob could manipulate a value of `u` such that if `e` is 1, the check passes even without knowing `x`.

## B.1 Generate `u` From `fx`

In order to fool Alice, Bob could calculate `u` as `g^r mod p / f(x)` instead of as `g^r mod p`.

This involves finding the multiplicative inverse of `fx mod p`.

For more on multiplicative inverse: https://en.wikipedia.org/wiki/Modular_multiplicative_inverse

We'll assume Bob is a powerful bad actor who can calculate this in order to attempt to fake the proof, though in practice this will not be easy for Bob.

In [11]:
def bad_actor_calculate_u(g, r, p, fx):
    fx_inverse = pow(fx, -1, p)
    u = (pow(g, r, p) * fx_inverse) % p
    
    return u

bad_u = bad_actor_calculate_u(g, r, p, fx)
bad_u

13

## B.2 Set `v` to `r`

Bob, the bad actor, can respond with `v = r`, instead of `v = r + x`, because their `bad_u` value will pass Alice's validation check regardless of the value of `x`.


In [12]:
def bad_actor_respond_to_challenge(r):
    v = r
    return v

## B.3 When `e` is 1, Bob Passes the Checks

When `e` is 1, Bob's `u` generated from `bad_actor_calculate_u()`, passes the `verify_proof()` checks (without needing to know `x`).

Note in the below code block that Bob is able to pass the checks despite never using `x` as an input to prepare what he submits to Alice. Neither `u` nor `v` incorporate `x` in any way, thus Bob is able to successfully lie and pass the check without actually needing to know `x`.

In [13]:
r = pick_private_random_int(p)
u = bad_actor_calculate_u(g, r, p, fx) # The bad actor function is used
e = 1 # Not randomizing `e` here to show Bob can successfully lie when `e` is 1
v = bad_actor_respond_to_challenge(r) # The bad actor function is used

verify_proof(e,u,g,v,p,fx)
    
print('No assert failures! Bob appears to be honest but he is a bad actor!')



No assert failures! Bob appears to be honest but he is a bad actor!


## B.4 If Bob Knows `e` Ahead of Time, He Passes All Checks

If Bob can successfully lie when `e` is 1, then why not just always use 0 for `e`?

Because if Bob *knows* that `e` will always be 0, he can prepare for that.

In fact, you may recall that earlier in Part A, Step 7, we asked the exact opposite question, "why bother ever using `e` of 0?"

As you can see, always using either `e` of 0, or `e` of 1, each has its own problems.

Bob can not be able to anticipate what `e` will be ahead of time, otherwise we can successfully lie.

Notice in the below code block, `e` is generated first, allowing Bob to react to how we submits a value of `u`. This simulates Bob knowing `e` ahead of time.

If Bob can guess `e` ahead of time, whether by luck or by a failure on Alice's part to sufficiently randomize `e`, then he can get away with lying regardless of the value of `e`.





In [14]:
for i in range(0,10):
    r = pick_private_random_int(p)
    e = generate_challenge()
    # Note how the challenge `e` is generated *before* Bob shares `u`, allowing him to tailor how he responds
    # Also note that Bob never needs to use `x` itself to prepare `u` or `v`, he can do this without knowing `x`
    if e == 0:
        u = calculate_u(g, r, p)
    else:
        u = bad_actor_calculate_u(g, r, p, fx)
        
    v = bad_actor_respond_to_challenge(r)
    
    verify_proof(e,u,g,v,p,fx)
    
print('No assert failures! Bob appears to be honest but he is a bad actor!')
    

No assert failures! Bob appears to be honest but he is a bad actor!


## B.5 Checks Must be Repeated, With `e` Shared After `u` for Bob to Fail

In order to be confident that we can catch Bob's lie, checks must be repeated multiple times, `e` must be randomized, and `u` must be shared *after* `e` is randomized.

Please note that the below cell *should have an AssertionError* this is the expected and desired outcome.

The AssertionError is indicating the proof verification failed when Bob was a bad actor and was lying about knowing `x`, which is exactly what we want to occur.


In [15]:
for i in range(0,10):
    r = pick_private_random_int(p)
    u = bad_actor_calculate_u(g, r, p, fx)
    e = generate_challenge()
    v = bad_actor_respond_to_challenge(r)
    verify_proof(e,u,g,v,p,fx)
    
print('No assert failures! Bob appears to be honest but he is a bad actor!')

AssertionError: Failed when e=0, we caught Bob lying about knowing x!

# Part C: A Non-Interactive Zero Knowledge Proof

So far in this tutorial, Alice has been able to verify, to a very high degree of confidence, that Bob does in fact know the value of `x`, without Bob having to reveal `x` itself.

In other words, Bob has been able to prove that he knows `x` while giving Alice "zero knowledge" about `x`.

However, the obvious drawback with the proof so far is that it requires a repeated back and forth between Bob and Alice (interactive proof). Ideally, Bob could create a proof that anyone could validate, without the need for going back and forth, or even directly communicating with Bob at all (non-interactive proof).

---

In the interactive case, repeating the proof multiple times with randomized challenges added an element that made it hard for a bad actor to predict how to fake an appropriate response.

The question then becomes, how do we retain this "hard to predict how to fake an appropriate response" trait of our proof, without relying on multiple random challenges as the source of that difficulty.

Instead of using repeated randomized challenges, we can use a hash to generate the challenge.

Since, as we already know, the outcome of a hash is extremely hard to predict given an input, a hash makes it effectively impossible for a bad actor to pre-calculate a response that will "game" the output of the hash.


In [16]:
import random
import hashlib

# Bob publishes f(x)
fx = pow(g, x, p)

# Bob picks random number r
r = random.randint(0, p - 1)

# Bob calculates u
u = pow(g, r, p)

# Bob calculates `e` as the hash of `u`
# Note how `e` is an extremely large number
# This makes it exceptionally difficult for a bad actor to find a `u` that hashes to a given `e`
e = int(hashlib.sha256(str(u).encode()).hexdigest(), 16)
e

66943969713086776032173464195394550745560523545004192271047021628614662158183

In [17]:
# Bob calculates `v`
v = (r + e * x) % (p - 1)

# Bob shares `v` and `u` with Alice

# Alice calculates her own `e` value by hashing `u`
e_alice = int(hashlib.sha256(str(u).encode()).hexdigest(), 16)
e_alice

66943969713086776032173464195394550745560523545004192271047021628614662158183

In [18]:
# Alice ensures that the values of `e` `u` and `v` satisfy the proof
assert pow(g, v, p) == (u * pow(fx, e_alice, p)) % p, "Failed, we caught Bob lying about knowing x!"

print('No assert failures! Bob is honest!')

No assert failures! Bob is honest!


With the above approach, Bob can prepare and publish the proof ahead of time, and anyone, at anytime, can verify the proof without multiple rounds required, and without ever needing to directly interact with Bob.

We have successfully modified our proof to be a non-interactive proof.

---

In theory, Bob could try to cheat by privately choosing different values of `u` until finding one that products a hash `e` which would pass the check, and then only share that `u` value he knows will work.

In practice however, this would essentially require a brute force attack that would be extremely difficult for two reasons.

1. The hash function H (e.g., SHA-256) produces outputs that appear random and are spread over a large range (from 0 to 2^256 - 1, to be precise). Therefore, Bob has no way to predict what `e` will be for a given `u`, and finding a suitable `u` by trial and error would take an enormous amount of time and computation power.

2. The proof verification that Alice performs is itself a one-way function. Even if Bob knows `e`, `g`, `p`, and `fx`, finding an `x` that makes the proof check hold true runs into the discrete logarithm problem we previously discussed. And without knowing `x`, Bob has no efficient way to find a suitable `v`, since `v` is directly dependent on the value of `x`.

Therefore, while it's theoretically possible for Bob to cheat by choosing a suitable `u`, in practice this would be computationally infeasible unless he knows `x` - which is exactly what the protocol is designed to verify in the first place.

---

A note on SNARKS:

At this point we now understand the core concepts of zero knowledge proofs, and have a working example of a very simplistic non-interactive zero knowledge proof. In practice, the (very) simple proof in this tutorial is still very far from being a SNARK, both in terms of complexity and performance.

Succinct Non-interactive ARguments of Knowledge (SNARKs) are a specific type of zero knowledge proof. 'Succinct' means the proofs are small in size and can be verified quickly. While it's technically possible to create a Python implementation of a SNARK, transforming our current tutorial into a SNARK is nontrivial would go well beyond basic Python and involves a deep understanding of cryptography techniques and number theory.

In a nutshell, to make a zero knowledge proof into a SNARK, we'd need to adopt cryptographic pairings and elliptic curve operations, or alternatively use arithmetic circuits, to create an argument system where the proof size is small (succinct) and the verification is fast.

For more about these topics: https://en.wikipedia.org/wiki/Elliptic-curve_cryptography



# Appendix

## I. Why does `p` need to be very large?

In our tutorial, we chose a `p` value for a simple demonstration of the core concepts. But in practice, if this were a real-world zero knowledge proof, `p` would need to be very large to ensure proper security.

In not, it would be easy to brute force attack (essentially guess `x` by trial-and-error), which means anyone could figure out what `x` is, which would undo the entire purpose of the proof in the first place: to keep `x` secret.

In the below, you can see just how quick and easy it would be for an attacker to figure out the secret value of `x` when using the actual `p` value we used in our tutorial.

In [19]:
%%time
def brute_force_method(fx, g, p):
    for x in range(p):
        print('Checking if x is equal to: ', x)
        if pow(g, x, p) == fx:
            print('Cracked the secret, x is equal to: ', x, )
            return x
    return None

brute_force_method(fx, g, p)

Checking if x is equal to:  0
Checking if x is equal to:  1
Checking if x is equal to:  2
Checking if x is equal to:  3
Checking if x is equal to:  4
Checking if x is equal to:  5
Checking if x is equal to:  6
Cracked the secret, x is equal to:  6
Wall time: 85.4 µs


6